# Lesson 08 — Microsoft Agent Framework: A2A Server

This notebook shows how to build an **A2A server** using **Microsoft Agent Framework** using your choice of model provider.

## What we build

```mermaid
flowchart TD
    LLM["AzureOpenAIChatClient\nGitHub Phi-4 or LocalFoundry"]
    LLM -->|".as_agent()"| Agent["Agent\nname · instructions · tools"]
    Agent --> Exec["PolicyQAExecutor\nAgentExecutor"]
    Exec --> Handler["DefaultRequestHandler\nInMemoryTaskStore"]
    Handler --> Server["A2AStarletteApplication\nport 10081"]
    Server -->|"A2A JSON-RPC"| Client["Any A2A Client"]
```

**Same use case as Lesson 05–07:** insurance policy Q&A assistant.

### Model provider options

| Provider                    | Variable value   | What you need                                                                 |
| --------------------------- | ---------------- | ----------------------------------------------------------------------------- |
| **GitHub Models**           | `"github"`       | `GITHUB_TOKEN` in `.env` — [get one free](https://github.com/settings/tokens) |
| **AI Toolkit LocalFoundry** | `"localfoundry"` | VS Code AI Toolkit extension, model loaded on port 5272                       |

Set `PROVIDER` in cell 2 before running.

Port used: **10081** (notebooks) · Full lesson script uses 10008


In [5]:
# ── Imports & environment setup ──────────────────────────────────
import os
import threading

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(raise_error_if_not_found=False))

# ── Model provider ────────────────────────────────────────────────
# "github"       — GitHub Models  (free, needs GITHUB_TOKEN in .env)
#                  https://github.com/settings/tokens
# "localfoundry" — AI Toolkit LocalFoundry  (local, no token needed)
#                  VS Code AI Toolkit → Models → Load a model → Run
PROVIDER = "github"  # ← change to "localfoundry" to use a local model

if PROVIDER == "github":
    ENDPOINT = "https://models.inference.ai.azure.com"
    API_KEY = os.environ.get("GITHUB_TOKEN", "")
    MODEL = "Phi-4"
    if not API_KEY:
        raise EnvironmentError("Set GITHUB_TOKEN in your .env or environment first")

elif PROVIDER == "localfoundry":
    ENDPOINT = "http://localhost:5272/v1/"
    API_KEY = "unused"  # LocalFoundry ignores the key
    MODEL = "Phi-4-mini-instruct"  # ← change to your loaded model ID
    print("ℹ  AI Toolkit LocalFoundry — ensure a model is running on port 5272")

else:
    raise ValueError(f"Unknown PROVIDER: {PROVIDER!r}")

SERVER_PORT = 10081

print(f"✓ Provider : {PROVIDER}")
print(f"✓ Endpoint : {ENDPOINT}")
print(f"✓ Model    : {MODEL}")
print(f"✓ Port     : {SERVER_PORT}")

✓ Provider : github
✓ Endpoint : https://models.inference.ai.azure.com
✓ Model    : Phi-4
✓ Port     : 10081


In [6]:
# ── Knowledge base (inline — no file path needed) ────────────────
POLICY = """
BRIGHT SHIELD INSURANCE — POLICY SUMMARY

Coverage Period:  January 1 – December 31 2025
Policy Number:    BS-2025-DEMO

DEDUCTIBLES
  Annual deductible (individual) : $500
  Annual deductible (family)     : $1,000
  Out-of-pocket maximum          : $3,500 / person

PREMIUMS
  Monthly premium (individual)   : $120
  Monthly premium (family)       : $340

COVERAGE
  Liability                      : $100,000 per incident
  Collision                      : Covered (ACV after deductible)
  Comprehensive                  : Covered (ACV after deductible)
  Rental car reimbursement       : Up to $35/day, 30-day maximum
  Emergency roadside assistance  : Included (unlimited calls)
  Medical payments               : $5,000 per person

EXCLUSIONS
  Intentional damage, racing, commercial use are NOT covered.

CLAIMS
  File within 30 days of incident.
  Contact: claims@brightshield.example  |  1-800-555-0199
"""

print("Policy document loaded.")
print(f"  Length: {len(POLICY)} characters")

Policy document loaded.
  Length: 911 characters


In [7]:
# ── Build the Microsoft Agent Framework agent ─────────────────────
#
# Pattern:
#   1. AzureOpenAIChatClient  — LLM connection (endpoint set by PROVIDER)
#   2. .as_agent()            — wraps client with name + instructions
#   3. await agent.run(msg)   — call the LLM

from agent_framework.azure import AzureOpenAIChatClient  # type: ignore[attr-defined]

INSTRUCTIONS = f"""\
You are a helpful insurance policy assistant for Bright Shield Insurance.
Answer questions accurately using the policy document provided.
If the answer is not in the document, say so clearly.
Keep answers concise — two or three sentences.

--- POLICY DOCUMENT ---
{POLICY}
--- END ---
"""

chat_client = AzureOpenAIChatClient(
    api_key=API_KEY,
    endpoint=ENDPOINT,
    deployment_name=MODEL,
)

agent = chat_client.as_agent(
    name="PolicyQAAgent",
    instructions=INSTRUCTIONS,
    tools=[],  # no tools — pure LLM Q&A
)

print(f"✓ Agent created : {agent.name}")
print(f"  Provider      : {PROVIDER}  ({MODEL})")

✓ Agent created : PolicyQAAgent
  Provider      : github  (Phi-4)


In [ ]:
# ── Quick smoke-test: call the agent directly ─────────────────────
#
# Jupyter already runs inside an event loop — use `await` directly
# rather than asyncio.run(), which raises RuntimeError in a notebook.

answer = await agent.run("What is the annual deductible for an individual?")
print("Q: What is the annual deductible for an individual?")
print(f"A: {answer}")

C:\Program Files\Python311\Lib\ast.py:50: RuntimeWarning: coroutine 'ask' was never awaited
  return compile(source, filename, mode, flags,


ChatClientException: <class 'agent_framework.azure._chat_client.AzureOpenAIChatClient'> service failed to complete the prompt: NOT FOUND

: 

In [ ]:
# ── Wrap the agent as an A2A server ──────────────────────────────
#
# Steps (same as Lesson 06 but using MAF instead of QAAgent):
#   1. Implement AgentExecutor.execute()  — calls agent.run()
#   2. Build AgentCard                    — describes the agent
#   3. Wire DefaultRequestHandler         — task lifecycle management
#   4. Build A2AStarletteApplication      — ASGI server

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.apps import A2AStarletteApplication
from a2a.server.events import EventQueue
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from a2a.utils import new_agent_text_message
from pydantic import AnyHttpUrl


class PolicyQAExecutor(AgentExecutor):
    """Bridges the A2A protocol to the MAF agent."""

    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        question = context.get_user_input()
        answer = await agent.run(question)
        await event_queue.enqueue_event(new_agent_text_message(str(answer)))

    async def cancel(self, context: RequestContext, event_queue: EventQueue) -> None:
        pass


agent_card = AgentCard(
    name="PolicyQAAgent",
    description="Insurance policy Q&A — Microsoft Agent Framework + GitHub Phi-4",
    url=AnyHttpUrl(f"http://localhost:{SERVER_PORT}/"),
    version="1.0.0",
    capabilities=AgentCapabilities(streaming=False),
    skills=[
        AgentSkill(
            id="policy_qa",
            name="Policy Q&A",
            description="Answer questions about the Bright Shield insurance policy",
            tags=["insurance", "qa"],
        )
    ],
)

handler = DefaultRequestHandler(
    agent_executor=PolicyQAExecutor(),
    task_store=InMemoryTaskStore(),
)

server_app = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)
app = server_app.build()

print("✓ A2A app built")
print(f"  Agent card : http://localhost:{SERVER_PORT}/.well-known/agent-card.json")
print(f"  JSON-RPC   : POST http://localhost:{SERVER_PORT}/")

In [ ]:
# ── Start the server in a background thread ──────────────────────
#
# The server runs in a daemon thread so the notebook stays interactive.
# Open client.ipynb to send A2A requests to this server.

import uvicorn


def _run_server():
    uvicorn.run(app, host="0.0.0.0", port=SERVER_PORT, log_level="warning")


server_thread = threading.Thread(target=_run_server, daemon=True)
server_thread.start()

import time

time.sleep(1.0)  # give uvicorn a moment to bind

import httpx

resp = httpx.get(
    f"http://localhost:{SERVER_PORT}/.well-known/agent-card.json", timeout=5
)
card = resp.json()
print(f"✓ Server live — status {resp.status_code}")
print(f"  Name        : {card.get('name')}")
print(f"  Description : {card.get('description')}")
print()
print("Now open client.ipynb to send A2A requests →")